# This file is used to flat the JSON data

In [1]:
import json
import pandas as pd
import numpy as np
from shapely.geometry import Point
from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5
import re
import math
import ast
from pandas.io.json import json_normalize
from flatten_json import flatten
import ProjectLookUpTable 
import socket, struct
from binascii import hexlify
import datetime
import macaddress
import os

# Read JSON data

In [2]:
def file_json_file_with_file_name(pcap_Json_file_name):
    json_data_file = open(pcap_Json_file_name, encoding='utf-8')
    file_data = json.load(json_data_file)
    json_data_file.close()
    return file_data

# type_map_for_json(json_record) map string value to int 

In [3]:
# type_map_for_json(json_record)


def IPV6_to_int(ipv6_addr):
    return int(hexlify(socket.inet_pton(socket.AF_INET6, ipv6_addr)), 16)

def IPV4_to_int(ip):
    packedIP = socket.inet_aton(ip)
    return struct.unpack("!L", packedIP)[0]

def type_map_for_json(json_record):
    if type(json_record) != dict:
        return True
    for json_key in json_record.keys():
        if type_map_for_json(json_record[json_key]):
            if json_key in ProjectLookUpTable.wire_shark_type_lookup_map:
                json_value_type = ProjectLookUpTable.wire_shark_type_lookup_map[json_key]
                json_str_value = json_record[json_key]
                # change number string to number 
                if isinstance(json_str_value, int) or isinstance(json_str_value, float):
                    json_record[json_key] = json_value_type
                elif json_value_type in ProjectLookUpTable.wire_shark_number_type_set:
                    try:
                        if '0x' in json_str_value:
                            json_record[json_key] = int(json_str_value, 0)
                        elif '.' in json_str_value:
                            json_record[json_key] = float(json_str_value)
                        else:
                            json_record[json_key] = int(json_str_value)
                    except:
                        json_record[json_key] = 0
                elif json_value_type in ProjectLookUpTable.wire_shark_ipaddr_type_set:
                    try:
                        if ':' in json_str_value:
                            json_record[json_key] = IPV6_to_int(json_str_value)
                        else:
                            json_record[json_key] = IPV4_to_int(json_str_value)
                    except:
                        json_record[json_key] = json_str_value
                elif json_value_type == 'Ethernet or other MAC address':
                    try:
                        json_record[json_key] = int(macaddress.MAC(json_str_value))
                    except:
                        json_record[json_key] = json_str_value
                # elif json_value_type == 'Date and time':
                #     json_str_value = json_str_value[0:json_str_value.index('.')+6]
                #     json_record[json_key] = datetime.datetime.strptime(json_str_value, '%b %d, %Y %H:%M:%S.%f')





In [4]:
def check_json_field_exist(json_record, local_wanted_packet_field_pair):
    if len(local_wanted_packet_field_pair) == 0:
        return True
    if local_wanted_packet_field_pair[0] in json_record:
        return local_wanted_packet_field_pair[1] == json_record[local_wanted_packet_field_pair[0]]
    rv = False
    for key in json_record:
        if isinstance(json_record[key], dict):
            rv = rv or check_json_field_exist(json_record[key], local_wanted_packet_field_pair)
    return rv


In [5]:
def get_wanted_packets(local_wanted_packets, local_json_record_list, local_wanted_packet_field_pair):
    rv = []
    idx = 0
    for record in local_json_record_list:
        if(idx < len(local_wanted_packet_field_pair)):
            paird_field = local_wanted_packet_field_pair[idx]
        else:
            paird_field = []
        if local_wanted_packets[idx] in record['_source']['layers'] and \
        check_json_field_exist(record['_source']['layers'][local_wanted_packets[idx]], paird_field):
            rv.append(record)
            idx += 1
            if idx >= len(local_wanted_packets):
                return rv
    return rv

# drop use less fields by use drop_json_fields(json_record, field_name_path_list)

In [6]:

def json_field_drop(json_record, path_list):
    if path_list[0] not in json_record:
        return
    if(len(path_list) == 1):
        del json_record[path_list.pop(0)]
    else:
        json_record = json_record[path_list.pop(0)]
        json_field_drop(json_record, path_list)

def drop_json_fields(json_record, field_name_path_list):
    for name_path_str in field_name_path_list:
        json_field_drop(json_record, name_path_str.split('->'))


In [7]:
def start_drop_field_value_mapping(local_json_records, local_json_drop_field_path_list, type_map = True):
    for local_json_record in local_json_records:
        local_json_record = local_json_record['_source']['layers']
        drop_json_fields(local_json_record, local_json_drop_field_path_list)
        if type_map:
            type_map_for_json(local_json_record)


In [8]:
def flatten_json_records(json_record_list):
    rv = []
    for record in json_record_list:
        record = record['_source']['layers']
        rv.append(flatten(record))
    return rv

# Only for Hancitor

In [9]:
from chepy import Chepy
def Hancitor_processing(local_key_field, target_http_records):
    contain_all = 0
    try:
        if local_key_field in target_http_records[2]:
            encode_str = target_http_records[2][local_key_field]
            url_set = []
            decode_str =str(Chepy(encode_str).base64_decode().xor(key="z", key_type="utf"))
            url_set.append(decode_str[decode_str.index('http://'):decode_str.index('}')])
            decode_str = decode_str[decode_str.index('}')+1:]
            url_set.append(decode_str[decode_str.index('http://'):decode_str.index('}')])
            print(url_set)
            for offSet in range(0,len(url_set)):
                if "http_http.request.full_uri" in target_http_records[3+offSet] \
                and target_http_records[3+offSet]["http_http.request.full_uri"] == url_set[offSet]:
                    contain_all += 1
            
            target_http_records[2]['Hancitor_url'] = contain_all
            
    except:
        target_http_records[2]['Hancitor_url'] = contain_all
        


# Create the string to integer map so the ML process can run faster 


In [10]:

def map_str_to_int(local_record_str_to_int_map, local_flatten_json_records_list):
    for flatten_json_record in local_flatten_json_records_list:
        for field_name in flatten_json_record:
            if isinstance(flatten_json_record[field_name], str):
                if flatten_json_record[field_name] not in local_record_str_to_int_map:
                    local_record_str_to_int_map[flatten_json_record[field_name]] = len(local_record_str_to_int_map)+1
                flatten_json_record[field_name] = local_record_str_to_int_map[flatten_json_record[field_name]]



In [11]:
def get_df_header_of_each_col(local_num_record, malware_json_records_list_list, normal_json_records_list_list):
    local_col_name_on_offset = {}

    # init set
    for off_set in range(local_num_record):
        local_col_name_on_offset[off_set] = set()

    # get all unique fiels of each records on each offset
    for local_json_records_list in malware_json_records_list_list:
        for off_set in range(local_num_record):
            if off_set < len(local_json_records_list):
                for json_field in local_json_records_list[off_set]:
                    local_col_name_on_offset[off_set].add(json_field)

    # get all unique fiels of each records on each offset
    for local_json_records_list in normal_json_records_list_list:
        for off_set in range(local_num_record):
            if off_set < len(local_json_records_list):
                for json_field in local_json_records_list[off_set]:
                    local_col_name_on_offset[off_set].add(json_field)

    return local_col_name_on_offset

In [12]:
 # Cast the list of json records into one record and load into panda
def make_list_json_record_into_one_record(local_col_name_on_offset, local_json_records_list_list, local_num_record):

    # use the offset and unique fields name to build df
    local_overall_field_list = []
    for off_set in range(local_num_record):
        for field_name in local_col_name_on_offset[off_set]:
            local_overall_field_list.append(field_name+'_'+str(off_set))
    local_df = pd.DataFrame(columns=local_overall_field_list)
    try:
        # load data into the df
        for local_json_records_list in local_json_records_list_list:
            local_record_array = []
            for off_set in range(local_num_record):
                if off_set < len(local_json_records_list):
                    local_json_record = local_json_records_list[off_set]
                    for df_col_field in local_col_name_on_offset[off_set]:
                        if df_col_field in local_json_record:
                            local_record_array.append(local_json_record[df_col_field])
                        else:
                            local_record_array.append(0)
            local_df.loc[len(local_df.index)] = local_record_array
    except:
        print(local_record_array)
    return local_df



# Main


In [13]:
# important variables 
json_records_list_list = []
file_name_lists = []

for top_file_path in os.listdir("../../data/URSNIF_GoziIFSB"):
    local_path = '../../data/URSNIF_GoziIFSB/'+top_file_path + '/only/only.json'
    if os.path.exists(local_path):
        file_name_lists.append(local_path)

wanted_packets = ['http', 'http', 'http', 'http', 'http', 'http']
wanted_packet_field_pair = []
json_drop_field_path_list = ['frame->frame.time', 'frame->frame.time_delta_displayed', 
    'frame->frame.coloring_rule.name', 'frame->frame.coloring_rule.string',
    'ip->ip.addr', 'ip->ip.host', 'tcp->tcp.port','http->http.request.line',
    'tcp->tcp.payload', 
    'ip->ip.src_host', 'ip->ip.dst_host', 'http->http.cookie_tree',  
    'tls->tls.record->tls.handshake->tls.handshake.random',
    'tls->tls.record->tls.handshake->tls.handshake.random_tree',
    'tls->tls.record->tls.handshake.ciphersuites',
]

record_str_to_int_map = {}

In [14]:
file_name_lists

['../../data/URSNIF_GoziIFSB/2020-03-31 -- Ursnif (GoziIFSB) infection/only/only.json',
 '../../data/URSNIF_GoziIFSB/2020-06-10 -- Ursnif (GoziIFSB) infection with Ursnif variant/only/only.json',
 '../../data/URSNIF_GoziIFSB/2020-07-07 -- Quick post Ursnif (GoziIFSB) with IcedID from English Word docs/only/only.json',
 '../../data/URSNIF_GoziIFSB/2020-07-09 -- Quick post Ursnif (GoziIFSB) from Italian Word docs/only/only.json',
 '../../data/URSNIF_GoziIFSB/2021-03-12 - TA551 (SHATHAK) ITALIAN TEMPLATE WORD DOCS PUSH URSNIF (GOZIISFB)/only/only.json',
 '../../data/URSNIF_GoziIFSB/2021-03-17 - TA551 (SHATHAK) ITALIAN TEMPLATE WORD DOCS PUSH URSNIF (GOZIISFB)/only/only.json',
 '../../data/URSNIF_GoziIFSB/2021-04-16 (FRIDAY) - TA551 (SHATHAK) GERMAN-TEMPLATE WORD DOCS PUSH URSNIF (GOZIISFB)/only/only.json',
 '../../data/URSNIF_GoziIFSB/2021-04-28 (WEDNESDAY) - TA551 (SHATHAK) PUSHES URSNIF (GOZIISFB)/only/only.json',
 '../../data/URSNIF_GoziIFSB/2021-06-18 (FRIDAY) - TA551 (SHATHAK) ENGLIS

In [15]:

for  pcap_json_file_name in file_name_lists: 
    # Read Json files
    json_record_list = file_json_file_with_file_name(pcap_json_file_name)
    # make sure the start
    json_record_list = get_wanted_packets(wanted_packets, json_record_list, wanted_packet_field_pair)
    # Drop not wanted fields 
    start_drop_field_value_mapping(json_record_list,json_drop_field_path_list)
    # Flate Json into field value map
    flatten_json_records_list = flatten_json_records(json_record_list)
    # Map String into int
    map_str_to_int(record_str_to_int_map, flatten_json_records_list)
    # save current file to list
    json_records_list_list.append(flatten_json_records_list)


In [16]:
json_records_list_list

[[{'frame_frame.encap_type': 1,
   'frame_frame.offset_shift': 0.0,
   'frame_frame.time_epoch': 1585680374.998239,
   'frame_frame.time_delta': 0.000444,
   'frame_frame.time_relative': 0.493439,
   'frame_frame.number': 6,
   'frame_frame.len': 500,
   'frame_frame.cap_len': 500,
   'frame_frame.marked': 0,
   'frame_frame.ignored': 0,
   'frame_frame.protocols': 1,
   'eth_eth.dst': 36168636208113,
   'eth_eth.dst_tree_eth.dst_resolved': 2,
   'eth_eth.dst_tree_eth.dst.oui': 2155818,
   'eth_eth.dst_tree_eth.dst.oui_resolved': 3,
   'eth_eth.dst_tree_eth.addr': 36168636208113,
   'eth_eth.dst_tree_eth.addr_resolved': 2,
   'eth_eth.dst_tree_eth.addr.oui': 2155818,
   'eth_eth.dst_tree_eth.addr.oui_resolved': 3,
   'eth_eth.dst_tree_eth.dst.lg': 0,
   'eth_eth.dst_tree_eth.lg': 0,
   'eth_eth.dst_tree_eth.dst.ig': 0,
   'eth_eth.dst_tree_eth.ig': 0,
   'eth_eth.src': 34395146158,
   'eth_eth.src_tree_eth.src_resolved': 4,
   'eth_eth.src_tree_eth.src.oui': 2050,
   'eth_eth.src_tree_

In [17]:
len(json_records_list_list)

11

# Read and output the normal traffic 

In [27]:
def get_normal_wanted_packets( local_wanted_packets, local_json_list, local_wanted_packet_field_pair):
    local_valid_record_List_group = []
    local_json_list_idx = 0
    while local_json_list_idx < len(local_json_list):
        if local_json_list_idx+len(local_wanted_packets)-1 >= len(local_json_list):
            break
        json_record = local_json_list[local_json_list_idx]
        if local_wanted_packets[0] in json_record['_source']['layers']:  # and local_wanted_packets[0] not in local_json_list[local_json_list_idx+1]['_source']['layers']
            local_valid_record_List_group.append(get_wanted_packets(wanted_packets, local_json_list[local_json_list_idx:],local_wanted_packet_field_pair))
            local_json_list_idx+=len(wanted_packets)
    return local_valid_record_List_group


In [28]:
normal_json_records_list_list = []

normal_file_name_lists = []
for top_file_path in os.listdir("../../data/URSNIF_normal/"):
    if top_file_path.endswith(".json"):
        local_path = '../../data/URSNIF_normal/'+top_file_path
        normal_file_name_lists.append(local_path)

normal_file_name_lists

['../../data/URSNIF_normal/only1.json',
 '../../data/URSNIF_normal/only10.json',
 '../../data/URSNIF_normal/only11.json',
 '../../data/URSNIF_normal/only2.json',
 '../../data/URSNIF_normal/only3.json',
 '../../data/URSNIF_normal/only4.json',
 '../../data/URSNIF_normal/only5.json',
 '../../data/URSNIF_normal/only6.json',
 '../../data/URSNIF_normal/only7.json',
 '../../data/URSNIF_normal/only8.json',
 '../../data/URSNIF_normal/only9.json']

In [29]:

for  pcap_json_file_name in normal_file_name_lists: 
    # Read Json files
    json_record_list = file_json_file_with_file_name(pcap_json_file_name)

    # make sure the start
    json_record_list_group = get_normal_wanted_packets(wanted_packets, json_record_list, wanted_packet_field_pair)

    for json_list in json_record_list_group:
        # Drop not wanted fields 
        start_drop_field_value_mapping(json_list,json_drop_field_path_list)
        # Flate Json into field value map
        flatten_json_records_list = flatten_json_records(json_list)
        if len(flatten_json_records_list) < 3:
            continue
        # For Hancitor, take the cookie and make them into fields 
        Hancitor_processing("http_http.file_data", flatten_json_records_list)
        # Map String into int
        map_str_to_int(record_str_to_int_map, flatten_json_records_list)
        # save current file to list
        normal_json_records_list_list.append(flatten_json_records_list)
    #break

In [30]:
len(normal_json_records_list_list)

468

# Output data table to CSV file 

In [31]:
col_name_on_offset = get_df_header_of_each_col(len(wanted_packets), json_records_list_list, []) 

In [32]:
# cast the IceId record into one record 
USNIF_df = make_list_json_record_into_one_record(col_name_on_offset, json_records_list_list, len(wanted_packets))
USNIF_df.to_csv('USNIF_only_data_set.csv', index=False)
USNIF_df

,eth_eth.dst_tree_eth.addr.oui_resolved_0,ip_ip.src_0,http_GET /iz5/yaca.php?l=kpt1.cab HTTP/1.1\r\n__ws.expert__ws.expert.message_0,http_GET /qlgmyrwmcf HTTP/1.1\r\n__ws.expert__ws.expert.severity_0,tcp_tcp.dstport_0,ip_ip.dsfield_tree_ip.dsfield.dscp_0,http_GET /bijol/30829/qxmbPmbPLTBgcHiQBTvpV3XkF/pMcV1KQoZvQXFyiQUQMM2RoYUyJaRoSyKnsQTWwia3/Td4YVmd8BZgkL2LfL0Q6rP5E90A4I/6uKdYjL3nxZvx0pbc3tbxwa6jMW1JDNxusaKQ52ftRS7D/Y1IPTRZnuRMLPgWYwLOsEAebOsxD7BBL/laka3?user=Z2O&sid=9s0FC1SUwC HTTP/1.1\r\n__ws.expert__ws.expert.message_0,eth_eth.src_0,eth_eth.src_tree_eth.ig_0,tcp_tcp.checksum.status_0,...,http_http.cache_control_5,frame_frame.ignored_5,http_http.response_for.uri_5,http_http.prev_response_in_5,tcp_tcp.ack_raw_5,http_http.connection_5,tcp_tcp.flags_tree_tcp.flags.syn_5,http_http.accept_5,http_http.content_length_header_5,ip_ip.flags_5
0,3.0,1.679768e+08,0.0,9.0,80.0,0.0,0.0,3.439515e+10,0.0,2.0,...,0.0,0.0,62.0,413.0,3.516193e+07,29.0,0.0,0.0,71.0,0.0
1,3.0,1.681680e+08,0.0,0.0,80.0,0.0,0.0,3.439515e+10,0.0,2.0,...,0.0,0.0,109.0,665.0,1.934910e+09,29.0,0.0,0.0,71.0,2.0
2,3.0,1.682328e+08,0.0,0.0,80.0,0.0,0.0,3.439515e+10,0.0,2.0,...,0.0,0.0,138.0,0.0,3.898002e+09,29.0,0.0,0.0,71.0,2.0
3,3.0,1.682333e+08,144.0,0.0,80.0,0.0,0.0,3.439515e+10,0.0,2.0,...,0.0,0.0,168.0,902.0,1.407589e+09,29.0,0.0,0.0,71.0,2.0
4,3.0,1.679719e+08,0.0,0.0,80.0,0.0,0.0,3.439515e+10,0.0,2.0,...,0.0,0.0,199.0,2873.0,2.249808e+09,29.0,0.0,0.0,71.0,0.0
5,3.0,1.679732e+08,0.0,0.0,80.0,0.0,0.0,3.439515e+10,0.0,2.0,...,0.0,0.0,232.0,1654.0,3.511771e+09,29.0,0.0,0.0,71.0,0.0
6,3.0,1.680385e+08,0.0,0.0,80.0,0.0,0.0,3.439515e+10,0.0,2.0,...,0.0,0.0,0.0,0.0,2.258759e+09,0.0,0.0,0.0,0.0,2.0
7,3.0,1.685659e+08,0.0,0.0,80.0,0.0,274.0,3.439515e+10,0.0,2.0,...,0.0,0.0,301.0,1233.0,3.865822e+08,29.0,0.0,0.0,71.0,2.0
8,3.0,1.681701e+08,0.0,0.0,80.0,0.0,0.0,3.439515e+10,0.0,2.0,...,0.0,0.0,0.0,0.0,5.790134e+07,29.0,0.0,39.0,0.0,2.0
9,3.0,1.683680e+08,0.0,0.0,80.0,0.0,0.0,3.439515e+10,0.0,2.0,...,355.0,0.0,369.0,0.0,5.344540e+08,92.0,0.0,0.0,373.0,0.0


In [33]:
# cast the normal record into one record 
normal_df = make_list_json_record_into_one_record(col_name_on_offset, normal_json_records_list_list, len(wanted_packets))
normal_df.to_csv('normal_USNIF_only_data_set.csv', index=False)
normal_df

,eth_eth.dst_tree_eth.addr.oui_resolved_0,ip_ip.src_0,http_GET /iz5/yaca.php?l=kpt1.cab HTTP/1.1\r\n__ws.expert__ws.expert.message_0,http_GET /qlgmyrwmcf HTTP/1.1\r\n__ws.expert__ws.expert.severity_0,tcp_tcp.dstport_0,ip_ip.dsfield_tree_ip.dsfield.dscp_0,http_GET /bijol/30829/qxmbPmbPLTBgcHiQBTvpV3XkF/pMcV1KQoZvQXFyiQUQMM2RoYUyJaRoSyKnsQTWwia3/Td4YVmd8BZgkL2LfL0Q6rP5E90A4I/6uKdYjL3nxZvx0pbc3tbxwa6jMW1JDNxusaKQ52ftRS7D/Y1IPTRZnuRMLPgWYwLOsEAebOsxD7BBL/laka3?user=Z2O&sid=9s0FC1SUwC HTTP/1.1\r\n__ws.expert__ws.expert.message_0,eth_eth.src_0,eth_eth.src_tree_eth.ig_0,tcp_tcp.checksum.status_0,...,http_http.cache_control_5,frame_frame.ignored_5,http_http.response_for.uri_5,http_http.prev_response_in_5,tcp_tcp.ack_raw_5,http_http.connection_5,tcp_tcp.flags_tree_tcp.flags.syn_5,http_http.accept_5,http_http.content_length_header_5,ip_ip.flags_5
0,434.0,3.232238e+09,0.0,0.0,80.0,0.0,0.0,1.981122e+14,0.0,2.0,...,476.0,0.0,469.0,0.0,3.332967e+09,15.0,0.0,0.0,475.0,0.0
1,434.0,3.232238e+09,0.0,0.0,80.0,0.0,0.0,1.981122e+14,0.0,2.0,...,476.0,0.0,735.0,44505.0,3.332967e+09,15.0,0.0,0.0,475.0,0.0
2,434.0,3.232238e+09,0.0,0.0,80.0,0.0,0.0,1.981122e+14,0.0,2.0,...,476.0,0.0,761.0,44534.0,3.332968e+09,15.0,0.0,0.0,721.0,0.0
3,434.0,3.232238e+09,0.0,0.0,80.0,0.0,0.0,1.981122e+14,0.0,2.0,...,476.0,0.0,787.0,44562.0,3.332969e+09,15.0,0.0,0.0,721.0,0.0
4,434.0,3.232238e+09,0.0,0.0,80.0,0.0,0.0,1.981122e+14,0.0,2.0,...,476.0,0.0,814.0,44590.0,3.332970e+09,15.0,0.0,0.0,818.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,689.0,1.745893e+09,0.0,0.0,54529.0,0.0,0.0,1.323840e+14,0.0,2.0,...,0.0,0.0,0.0,0.0,3.846740e+09,15.0,0.0,17.0,0.0,2.0
464,689.0,1.750820e+09,0.0,0.0,65146.0,0.0,0.0,1.323840e+14,0.0,2.0,...,0.0,0.0,0.0,0.0,1.394858e+09,15.0,0.0,17.0,0.0,2.0
465,689.0,2.955511e+09,0.0,0.0,65173.0,0.0,0.0,1.323840e+14,0.0,2.0,...,0.0,0.0,0.0,0.0,1.373886e+09,15.0,0.0,17.0,0.0,2.0
466,689.0,1.753196e+09,0.0,0.0,65250.0,0.0,0.0,1.323840e+14,0.0,2.0,...,0.0,0.0,0.0,0.0,1.098387e+09,15.0,0.0,17.0,0.0,2.0


In [34]:
# output flated data and str map to files
with open('USNIF_record_str_to_int_map.json', 'w') as fp:
    json.dump(record_str_to_int_map, fp)

# Free and testing space, do not run the following 

In [ ]:
wire_shark_type = set()
for key,val in ProjectLookUpTable.wire_shark_type_lookup_map.items():
    wire_shark_type.add(val)
wire_shark_type

In [ ]:

wire_shark_type_lookup_map = {}
wire_shark_description_lookup_map = {}
for i in range(len(field_list)):
    wire_shark_type_lookup_map[field_list[i]] = type_list[i]
    wire_shark_description_lookup_map[field_list[i]] = description_list[i]
